In [ ]:
'''
        # Plot accuracy curves
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=list(range(len(history['accuracy']))),
                                y=history['accuracy'],
                                mode='lines',
                                name='Train Accuracy'))
        fig.add_trace(go.Scatter(x=list(range(len(history['val_accuracy']))),
                                y=history['val_accuracy'],
                                mode='lines',
                                name='Validation Accuracy'))
        fig.update_layout(title='Accuracy Curves',
                        xaxis_title='Epochs',
                        yaxis_title='Accuracy')
        fig.update_layout({"template":"plotly_dark"})


        # Display the accuracy plot
        st.plotly_chart(fig)

        # Plot loss curves
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=list(range(len(history['loss']))),
                                y=history['loss'],
                                mode='lines',
                                name='Train Loss'))
        fig.add_trace(go.Scatter(x=list(range(len(history['val_loss']))),
                                y=history['val_loss'],
                                mode='lines',
                                name='Validation Loss'))
        fig.update_layout(title='Loss Curves',
                        xaxis_title='Epochs',
                        yaxis_title='Loss')
        fig.update_layout({"template":"plotly_dark"})


        # Display the loss plot
        st.plotly_chart(fig) '''

## anove is code to make loses in plotly

import streamlit as st
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions, InceptionV3
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import pickle
import streamlit.components.v1 as components
from skimage.feature import greycomatrix, greycoprops
from sklearn.preprocessing import StandardScaler
import joblib


scaler = StandardScaler()

# Everything u write here goes to "tab name"
st.set_page_config(
    page_title = "Breast Cancer Predictor",
    page_icon = "🎗️")

# Use local CSS
def local_css(file_name):
    with open(file_name) as f:
        st.markdown(f"<style>{f.read()}</style>", unsafe_allow_html=True)
local_css(r"C:\Users\b\Desktop\Sem 4\Breast Cancer\style\style.css")

def ChangeWidgetFontSize(wgt_txt, wch_font_size = '12px'):
    htmlstr = """<script>var elements = window.parent.document.querySelectorAll('p'), i;
                for (i = 0; i < elements.length; ++i) 
                    { if (elements[i].textContent.includes(|wgt_txt|)) 
                        { elements[i].style.fontSize ='""" + wch_font_size + """'; } }</script>  """

    htmlstr = htmlstr.replace('|wgt_txt|', "'" + wgt_txt + "'")
    components.html(f"{htmlstr}", height=0, width=0)


# Load Animation
animation_symbol = "🎗️"

st.markdown(
    f"""
    <div class="snowflake">{animation_symbol}</div>
    <div class="snowflake">{animation_symbol}</div>
    <div class="snowflake">{animation_symbol}</div>
    <div class="snowflake">{animation_symbol}</div>
    <div class="snowflake">{animation_symbol}</div>
    <div class="snowflake">{animation_symbol}</div>
    """,
    unsafe_allow_html=True,
)

def get_glcm_feature(image):
    # Compute the gray-level co-occurrence matrix (GLCM)
    glcm = greycomatrix(image, [1], [0], levels=256, symmetric=True, normed=True)
    # Compute the contrast, dissimilarity, homogeneity, ASM, energy, and correlation from the GLCM
    contrast = greycoprops(glcm, 'contrast')[0][0]
    dissimilarity = greycoprops(glcm, 'dissimilarity')[0][0]
    homogeneity = greycoprops(glcm, 'homogeneity')[0][0]
    ASM = greycoprops(glcm, 'ASM')[0][0]
    energy = greycoprops(glcm, 'energy')[0][0]
    correlation = greycoprops(glcm, 'correlation')[0][0]
    return [contrast, dissimilarity, homogeneity, ASM, energy, correlation]



st.title("Breast Cancer Prediction!")

model = tf.keras.models.load_model(r"C:\Users\b\Desktop\Sem 4\Breast Cancer\tensorflow_model.h5")

#model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


def import_and_predict(image_Data, model):
    # Preprocess the input image using InceptionV3
    img = cv2.resize(image_Data, (224, 224))
    x = preprocess_input(np.expand_dims(img, axis=0))
    
    # Make predictions using the loaded TensorFlow model
    predictions = model.predict(x)
    return predictions

def generate_pie_chart(labels, values, colors):
    fig = px.pie(names=labels, values=values)
    fig.update_traces(marker=dict(colors=colors))
    return fig

def load_model(model_path):
    with open(model_path, 'rb') as file:
        data = pickle.load(file)
    return data


file = st.file_uploader("Please upload an image", type =["jpg", "jpeg", "png"])

if file is None:
    st.text("Please upload an image")
else:

    listTabs = ['Recommended','For Advanced Users']
    tab1, tab2 = st.tabs(listTabs)
    ChangeWidgetFontSize(listTabs[0], '19px')
    ChangeWidgetFontSize(listTabs[1], '19px')

    with tab1:

        button_clicked = st.button("Evaluate")

        # Check if the button has been clicked
        if button_clicked:

            image = Image.open(file).convert('RGB') 
            open_cv_image = np.array(image) 
            # Convert RGB to BGR 
            image = open_cv_image[:, :, ::-1].copy() 
            st.image(image, use_column_width = True)
            predictions = import_and_predict(image, model)
            #st.success(predictions)

            if predictions[0][0] > predictions[0][1]:
                st.subheader("The image is predicted to be: **Cancerous** (Benign)  with {:.2f}% probability.".format(predictions[0][0]*100))
            elif predictions[0][1] > predictions[0][2]:
                st.subheader("The image is predicted to be: **Cancerous** (Malignant)  with {:.2f}% probability.".format(predictions[0][1]*100))
            else:
                st.subheader("The image is predicted to be: **Non-Cancerous** (Normal)  with {:.2f}% probability.".format(predictions[0][2]*100))

            labels = ['Benign', 'Malignant', 'Normal']
            values = predictions[0]
            colors = ['#FEBA4F', '#ff3166', '#00A693']

            st.plotly_chart(generate_pie_chart(labels, values, colors))


            with open(r"C:\Users\b\Desktop\Sem 4\Breast Cancer Code\Source Code\history_inception.pkl", 'rb') as f:
                history = pickle.load(f)

            with st.expander("Click To Display Accuracy And Losss Of The Model"):

                fig, ax = plt.subplots()
                ax.plot(history['accuracy'], label='train_accuracy')
                ax.plot(history['val_accuracy'], label='val_accuracy')
                ax.set_xlabel('Epochs')
                ax.set_ylabel('Accuracy')
                ax.legend()
                st.pyplot(fig)

                fig, ax = plt.subplots()
                ax.plot(history['loss'], label='train_loss')
                ax.plot(history['val_loss'], label='val_loss')
                ax.set_xlabel('Epochs')
                ax.set_ylabel('Loss')
                ax.legend()
                st.pyplot(fig)


    with tab2:
        # Dictionary to store the model names and their corresponding preprocessing techniques
        models = {
            'SVM (OTSU thresholding)': r'C:\Users\b\Desktop\Model\SVM_OTSU.pkl',
            'PCA+SVM (OTSU thresholding)': r'C:\Users\b\Desktop\Model\SVM_PCA_OTSU.pkl',
            'LDA+SVM (OTSU thresholding)': r'C:\Users\b\Desktop\Model\SVM_LDA_OTSU.pkl',
            'Decision Tree (OTSU thresholding)': r'C:\Users\b\Desktop\Model\DT_OTSU.pkl',
            'Random Forest (OTSU thresholding)': r'C:\Users\b\Desktop\Model\RF_OTSU.pkl',
            'Random Forest (GLCM feature extraction and scaling)': r'C:\Users\b\Desktop\Model\RF_GLCM.pkl',
            'SVM (GLCM feature extraction and scaling)': r'C:\Users\b\Desktop\Model\SVM_GLCM.pkl',
            'Inception V3': 'inception_v3.joblib'
        }

        model_choice = st.selectbox("Select a model", list(models.keys()))

        # Load the selected model
        model_path = models[model_choice]

        button_clicked = st.button("Run")

        # Check if the button has been clicked
        if button_clicked:

            if model_path == r"C:\Users\b\Desktop\Model\SVM_OTSU.pkl":

                model = load_model(model_path)

                image = Image.open(file)
                image = image.convert('L')

                image = np.array(image).copy()

                glcm_features = get_glcm_feature(image)
                X = scaler.fit_transform([glcm_features])
                st.write(X)

                # Make a prediction using the SVM classifier
                prediction = model.predict(X)
                st.write(prediction)

                if prediction == 0:
                    st.subheader("Prediction: Benign")
                elif prediction == 1:
                    st.subheader("Prediction: Malignant")
                else:
                    st.subheader("Prediction: Normal")


            if model_path == r"C:\Users\b\Desktop\Model\SVM_GLCM.pkl":

                model = load_model(model_path)

                image = Image.open(file)
                st.image(image, caption='Uploaded Image', use_column_width=True)
                
                # Extract features from the image
                features = get_glcm_feature(image)
                
                # Preprocess the data by scaling the features
                scaler = joblib.load(r"C:\Users\b\Desktop\Model\SVM_GLCM_scaler.pkl")
                features_scaled = scaler.transform(features)
                features_scaled = features_scaled.copy()
                
                # Make a prediction using the trained Random Forest model
                prediction = model.predict(features_scaled)[0]
                st.write(prediction)
                '''
                proba = model.predict_proba(features_scaled)[0][1]
                
                # Display the prediction and probability score
                if prediction == 0:
                    st.write('The tissue is normal with a probability score of {:.2f}%'.format((1-proba)*100))
                else:
                    st.write('The tissue is malignant with a probability score of {:.2f}%'.format(proba*100))'''


            if model_path == r"C:\Users\b\Desktop\Model\RF_GLCM.pkl":

                model = load_model(model_path)

                image = Image.open(file)
                image = image.convert('L')

                image = np.array(image).copy()

                glcm_features = get_glcm_feature(image)
                X = scaler.fit_transform([glcm_features])
                st.write(X)

                # Make a prediction using the SVM classifier
                prediction = model.predict(X)
                st.write(prediction)

                if prediction == 0:
                    st.subheader("Prediction: Benign")
                elif prediction == 1:
                    st.subheader("Prediction: Malignant")
                else:
                    st.subheader("Prediction: Normal")
        
        


In [ ]:
ValueError: buffer source array is read-only
Traceback:
File "C:\Users\b\AppData\Local\Programs\Python\Python310\lib\site-packages\streamlit\runtime\scriptrunner\script_runner.py", line 565, in _run_script
    exec(code, module.__dict__)
File "C:\Users\b\Desktop\Sem 4\Breast Cancer Code\Source Code\Pages\4 Breast Cancer Predicton.py", line 216, in <module>
    features = get_glcm_feature(image)
File "C:\Users\b\Desktop\Sem 4\Breast Cancer Code\Source Code\Pages\4 Breast Cancer Predicton.py", line 59, in get_glcm_feature
    glcm = greycomatrix(image, [1], [0], levels=256, symmetric=True, normed=True)
File "C:\Users\b\AppData\Local\Programs\Python\Python310\lib\site-packages\skimage\_shared\utils.py", line 480, in wrapped
    return func(*args, **kwargs)
File "C:\Users\b\AppData\Local\Programs\Python\Python310\lib\site-packages\skimage\feature\__init__.py", line 38, in greycomatrix
    return graycomatrix(image, distances, angles, levels, symmetric, normed)
File "C:\Users\b\AppData\Local\Programs\Python\Python310\lib\site-packages\skimage\feature\texture.py", line 142, in graycomatrix
    _glcm_loop(image, distances, angles, levels, P)
File "skimage\feature\_texture.pyx", line 19, in skimage.feature._texture._glcm_loop
File "stringsource", line 658, in View.MemoryView.memoryview_cwrapper
File "stringsource", line 349, in View.MemoryView.memoryview.__cinit__